## Imports

In [1]:
import copy
import numpy as np
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
from config import config
import os
import sys
current_dir = os.getcwd()
path = "C:\\Users\\eirik\\Documents\\Master\\ISLBBNN\\islbbnn"
# path = "C:\\you\\path\\to\\islbbnn\\folder\\here"
os.chdir(path)
import plot_functions as pf
import pipeline_functions as pip_func
sys.path.append('networks')
from flow_sigmoid_net import BayesianNetwork

os.chdir(current_dir) # set the working directory back to this one 

CPUs are used!


# Attain data

Problem:

$$y = x_1 + x_2 + x_1\cdot x_2 + x_1^2 + x_2^2 + 100 +\epsilon$$

where $\epsilon \sim N(0,0.01)$. 


Can make $x_3$ dependent on $x_1$. The depedence is defined in the following way:

\begin{align*}
 x_1 &\sim Unif(-10,10) \\
 x_3 &\sim Unif(-10,10) \\
 x_3 &= \text{dep}\cdot x_1 + (1-\text{dep})\cdot x_3
\end{align*}

## Pre process and batch size

In [2]:
# define parameters
HIDDEN_LAYERS = config['n_layers'] - 2 
epochs = config['num_epochs']
post_train_epochs = config['post_train_epochs']
dim = config['hidden_dim']
num_transforms = config['num_transforms']
n_nets = config['n_nets']
n_samples = config['n_samples']
lr = config['lr']
class_problem = config["class_problem"]
non_lin = config["non_lin"]
verbose = config['verbose']
save_res = config['save_res']
patience = config['patience']
SAMPLES = 1


# Get linear data, here a regression problem
y, X = pip_func.create_data_unif(n_samples, beta=[100,1,1,1,1], dep_level=0.0, classification=class_problem, non_lin=non_lin)

n, p = X.shape  # need this to get p 
print(n,p,dim)

# Define BATCH sizes
BATCH_SIZE = int((n*0.8)/100)
TEST_BATCH_SIZE = int(n*0.10) # Would normally call this the "validation" part (will be used during training)
VAL_BATCH_SIZE = int(n*0.10) # and this the "test" part (will be used after training)

TRAIN_SIZE = int((n*0.80)/100)
TEST_SIZE = int(n*0.10) # Would normally call this the "validation" part (will be used during training)
VAL_SIZE = int(n*0.10) # and this the "test" part (will be used after training)

NUM_BATCHES = TRAIN_SIZE/BATCH_SIZE

print(NUM_BATCHES)

assert (TRAIN_SIZE % BATCH_SIZE) == 0
assert (TEST_SIZE % TEST_BATCH_SIZE) == 0

40000 4 10
1.0


## Seperate a test set for later

In [3]:
# Split keep some of the data for validation after training
X, X_test, y, y_test = train_test_split(
    X, y, test_size=0.10, random_state=42)#, stratify=y)

test_dat = torch.tensor(np.column_stack((X_test,y_test)),dtype = torch.float32)

# Train network

## Device setup

In [4]:
# select the device and initiate model

# DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "mps")
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
LOADER_KWARGS = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}

## Train, validate, and test network

In [5]:
all_nets = {}
metrics_several_runs = []
metrics_median_several_runs = []
for ni in range(n_nets):
    post_train = False
    print('network', ni)
    # Initate network
    torch.manual_seed(ni+42)
    net = BayesianNetwork(dim, p, HIDDEN_LAYERS, classification=class_problem, num_transforms=num_transforms).to(DEVICE)
    alphas = pip_func.get_alphas_numpy(net)
    nr_weights = np.sum([np.prod(a.shape) for a in alphas])
    print(nr_weights)

    optimizer = optim.Adam(net.parameters(), lr=lr)
    
    all_nll = []
    all_loss = []

    # Split into training and test set
    X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=1/9, random_state=ni)#, stratify=y)
            
    train_dat = torch.tensor(np.column_stack((X_train,y_train)),dtype = torch.float32)
    val_dat = torch.tensor(np.column_stack((X_val,y_val)),dtype = torch.float32)
    
    # Train network
    counter = 0
    highest_acc = 0
    best_model = copy.deepcopy(net)
    for epoch in range(epochs + post_train_epochs):
        if verbose:
            print(epoch)
        nll, loss = pip_func.train(net, train_dat, optimizer, BATCH_SIZE, NUM_BATCHES, p, DEVICE, nr_weights, post_train=post_train)
        nll_val, loss_val, ensemble_val = pip_func.val(net, val_dat, DEVICE, verbose=verbose, reg=(not class_problem), post_train=post_train)
        if ensemble_val >= highest_acc:
            counter = 0
            highest_acc = ensemble_val
            best_model = copy.deepcopy(net)
        else:
            counter += 1
        
        all_nll.append(nll)
        all_loss.append(loss)

        if epoch == epochs-1:
            post_train = True   # Post-train --> use median model 
            for name, param in net.named_parameters():
                for i in range(HIDDEN_LAYERS+1):
                    #if f"linears{i}.lambdal" in name:
                    if f"linears.{i}.lambdal" in name:
                        param.requires_grad_(False)

        if counter >= patience:
            break
        
    all_nets[ni] = net 
    # Results
    metrics, metrics_median = pip_func.test_ensemble(all_nets[ni],test_dat,DEVICE,SAMPLES=10, reg=(not class_problem)) # Test same data 10 times to get average 
    metrics_several_runs.append(metrics)
    metrics_median_several_runs.append(metrics_median)
    pf.run_path_graph(all_nets[ni], threshold=0.5, save_path=f"path_graphs/flow/prob/test{ni}", show=verbose)

if verbose:
    print(metrics)
m = np.array(metrics_several_runs)
m_median = np.array(metrics_median_several_runs)

network 0
54
0
loss 642.6329956054688
nll 102.93635559082031
density 0.7308591364158524

val_loss: 1514.6112, val_nll: 970.3116, val_ensemble: 0.9157, used_weights_median: 38

1
loss 280.2561340332031
nll 67.78437805175781
density 0.3440176741116577

val_loss: 907.3023, val_nll: 716.1975, val_ensemble: 0.9360, used_weights_median: 12

2
loss 202.9445037841797
nll 77.06227111816406
density 0.2888615510116021

val_loss: 894.8884, val_nll: 773.8152, val_ensemble: 0.9032, used_weights_median: 12

3
loss 234.55001831054688
nll 136.05116271972656
density 0.2760129857428924

val_loss: 686.0006, val_nll: 583.4470, val_ensemble: 0.9307, used_weights_median: 12

4
loss 123.85497283935547
nll 51.23870849609375
density 0.2479298677623134

val_loss: 807.9278, val_nll: 719.6821, val_ensemble: 0.9153, used_weights_median: 12

5
loss 98.47789001464844
nll 37.98320770263672
density 0.23213916607912527

val_loss: 729.1021, val_nll: 676.9336, val_ensemble: 0.9137, used_weights_median: 12

6
loss 94.54295

In [6]:
pf.run_path_graph_weight(net, save_path="path_graphs/flow/weight/temp", show=True)

In [7]:
pip_func.get_alphas_numpy(net)

[array([[9.9999964e-01, 9.9996090e-01, 1.5115531e-04, 2.5657828e-03],
        [2.6851173e-03, 2.9355574e-03, 1.4375076e-03, 2.6071442e-03],
        [2.2750490e-03, 2.4024451e-03, 1.8528900e-03, 2.4174824e-03],
        [2.9703800e-03, 2.4883435e-03, 2.2292605e-03, 2.1040537e-03],
        [9.9999809e-01, 2.3150647e-03, 2.3487190e-04, 2.0145923e-03],
        [2.5630447e-03, 3.0301993e-03, 1.4855900e-03, 2.6956520e-03],
        [2.7008834e-03, 2.8946330e-03, 1.4668647e-03, 2.5904337e-03],
        [2.6486372e-03, 3.2695218e-03, 1.3412469e-03, 2.6074641e-03],
        [9.9997437e-01, 9.9998975e-01, 1.7341014e-04, 1.8760621e-03],
        [2.5329755e-03, 9.9999666e-01, 1.6356248e-04, 2.1925431e-03]],
       dtype=float32),
 array([[9.9999940e-01, 2.1541100e-05, 2.0293974e-05, 2.9557488e-05,
         9.9999881e-01, 5.8485182e-07, 4.1964936e-06, 2.1145074e-06,
         9.9999917e-01, 9.9999988e-01, 9.9999154e-01, 9.9999535e-01,
         6.7507744e-06, 1.9013408e-03]], dtype=float32)]

In [8]:
pip_func.weight_matrices_numpy(net)

[array([[ 9.3055773e-01,  8.4735668e-01,  8.0786740e-05, -1.5375133e-02],
        [ 1.4762272e-01,  7.9027258e-02,  6.3829917e-01,  6.4444321e-01],
        [-3.2633621e-02, -7.1632033e-03, -7.2261584e-01, -1.0276437e-01],
        [ 6.9746897e-02,  5.1625051e-02,  2.1614257e-02,  2.8378332e-01],
        [ 7.6131731e-01,  1.6784146e-03,  3.4751797e-03,  9.1910101e-03],
        [ 3.7883043e-01,  1.3904580e-01, -4.1000649e-01,  1.8288944e+00],
        [ 2.3599084e-01,  1.2175232e-01,  3.8661119e-01,  6.2481534e-01],
        [ 3.6576095e-03,  1.1774292e-02, -2.3552596e-03,  3.6518177e-01],
        [-6.4198449e-02,  7.6945370e-01, -4.5933020e-03, -4.3603316e-02],
        [ 7.9823658e-03,  7.3902744e-01, -9.6863206e-04,  7.8433268e-03]],
       dtype=float32),
 array([[-4.37152052e+00,  1.74504844e-03, -2.56697871e-02,
          2.92032049e-03,  4.54187250e+00,  6.65070684e-06,
          8.74994993e-02,  8.64031899e-05,  2.78689027e+00,
          2.36583805e+00, -8.49775434e-01, -3.79067004e-